In [ ]:
import sympy as sym

# Define the symbols
# mx, my, mz = sym.symbols('mx my mz')
# hx, hy, hz = sym.symbols('hx hy hz')
mu0, Kv, Ks, Ms, J1, J2 = sym.symbols('mu_0 K_v K_s M_s J_1, J_2')
theta_m1, phi_m1 = sym.symbols('theta_1 phi_1')
theta_m2, phi_m2 = sym.symbols('theta_2 phi_2')
theta_h, phi_h = sym.symbols('theta_h phi_h')
hx, hy, hz = sym.symbols('h_x h_y h_z')
inplane_angle = sym.symbols('alpha')
alpha = [sym.cos(inplane_angle), sym.sin(inplane_angle), 0]
m1 = sym.Matrix([
    sym.sin(theta_m1) * sym.cos(phi_m1),
    sym.sin(theta_m1) * sym.sin(phi_m1),
    sym.cos(theta_m1)
])
m2 = sym.Matrix([
    sym.sin(theta_m2) * sym.cos(phi_m2),
    sym.sin(theta_m2) * sym.sin(phi_m2),
    sym.cos(theta_m2)
])

h = sym.Matrix([
    hx, hy, hz
    # sym.sin(theta_h) * sym.cos(phi_h),
    # sym.sin(theta_h) * sym.sin(phi_h),
    # sym.cos(theta_h)
])
Hdemag = sym.Matrix([0, 0, -m1[2] * Ms])

field_energy = -mu0 * Ms * m1.dot(h)
demagnetization_energy = -(1. / 2.) * mu0 * Hdemag.dot(m1)
surface_anistropy = (-Ks + (1. / 2.) * mu0 * (Ms**2)) * m1[-1]
volume_anisotropy = -Kv * m1.dot(alpha)
iec_energy = -J1 * m1.dot(m2)
iec_quadratic_energy = -J2 * (m1.dot(m2))**2

total_energy = field_energy + volume_anisotropy + surface_anistropy + iec_energy + iec_quadratic_energy

# Define the magnetic field
# H = sym.Matrix([hx, hy, hz])
# m = sym.Matrix([mx, my, mz])
total_energy

In [ ]:
sym.print_latex(total_energy)

In [ ]:
for energy_tensor, label in zip([
        field_energy, volume_anisotropy, surface_anistropy, iec_energy,
        iec_quadratic_energy
], ["field", "volume", "surface", "iec", "quad iec"]):
    print(f"\n{label}")
    sym.print_latex(sym.diff(energy_tensor, theta_m1))
    sym.print_latex(sym.diff(energy_tensor, phi_m1))

In [ ]:
first_derivatives = {}
second_derivatives = {}
dangle = (theta_m1, phi_m1)
names = ('dtheta', 'dphi')
for energy_tensor, label in zip([
        field_energy, volume_anisotropy, surface_anistropy, iec_energy,
        iec_quadratic_energy
], ["field", "volume", "surface", "iec", "quad iec"]):
    for name, grad in zip(names, dangle):
        first_derivatives[f"{label} {name}"] = sym.diff(energy_tensor, grad)
        for name2, grad2 in zip(names, dangle):
            second_derivatives[f"{label} {name} {name2}"] = sym.diff(
                first_derivatives[f"{label} {name}"], grad2)


In [ ]:
for k, v in second_derivatives.items():
    print(f"\n{k}")
    sym.print_latex(v)

In [ ]:
first_derivatives = {}
second_derivatives = {}
dangle = (theta_m1, phi_m1)
for grad in dangle:
    first_derivatives[grad] = sym.diff(total_energy, grad)
    for grad2 in dangle:
        second_derivatives[(grad, grad2)] = sym.diff(first_derivatives[grad],
                                                     grad2)

In [ ]:
for der, val in second_derivatives.items():
    print("\nDERIVATIVE OF ENERGY WITH RESPECT TO", der)
    print(sym.print_latex(sym.simplify(val)))

# Numerical tests

In [ ]:
from cmtj.models.sb import *
from cmtj.utils import mu0
import matplotlib.pyplot as plt
import numpy as np

layer = LayerSB(
    thickness=1e-9,
    m=VectorObj(np.deg2rad(89), np.deg2rad(0)),
    Kv=VectorObj(np.deg2rad(3), np.deg2rad(45), 1e4),
    Ks=3e3,
    Ms=1,
)
model = SmitBeljersModel(layers=[layer],
                         eta=1e-2,
                         Hext=VectorObj(np.deg2rad(45),
                                        np.deg2rad(15),
                                        mag=100e3))

# model.gradient_descent(max_steps=1e9, tol=1e-2)
model.adam_gradient_descent(max_steps=1e9, tol=1e-8, learning_rate=1e-4)
with plt.style.context(['science', 'nature']):
    fig, ax = plt.subplots(dpi=300)
    ax2 = ax.twinx()
    ax2.plot(model.history['energy_0'], color='lavender')
    ax.plot(np.rad2deg(model.history['theta_0']),
            color='crimson',
            label=r'$\theta$')
    ax.plot(np.rad2deg(model.history['phi_0']),
            color='royalblue',
            label=r'$\phi$')

    ax.set_xlabel("Iteration")
    ax.set_ylabel("Angle ($^\circ$)")
    ax2.set_ylabel("Energy ($\mu J/m^3$)")
    ax.legend()

In [ ]:
with plt.style.context(['science', 'nature']):
    fig, ax = plt.subplots(dpi=300)
    # for label in model.ener_labels:
        # ax.plot(model.history[f'energy_{label}_0'], label=label)
    ax.plot(model.history['energy_volume_0'], label='total')
    ax.set_ylabel("Energy ($\mu J/m^3$)")
    ax.legend()

In [1]:
from cmtj.models.sb import *
from cmtj.utils import mu0
import matplotlib.pyplot as plt
from tqdm import tqdm 
import numpy as np

layerA = LayerSB(
    thickness=1e-9,
    m=VectorObj(np.deg2rad(89), np.deg2rad(10)),
    Kv=VectorObj(np.deg2rad(0), np.deg2rad(90), 1e3),
    Ks=30e3,
    Ms=1.2,
)
layerB = LayerSB(
    thickness=1e-9,
    m=VectorObj(np.deg2rad(89), np.deg2rad(10)),
    Kv=VectorObj(np.deg2rad(0), np.deg2rad(90), 1e6),
    Ks=0,
    Ms=1.3,
)
ms, fs = [], []
Hspace = np.linspace(-800e3, 800e3, 50)
for Hmag in tqdm(Hspace, desc="Field scan"):
    model = SmitBeljersModel(layers=[layerA, layerB],
                             J=[1e-6],
                             eta=1e-2,
                             Hext=VectorObj(np.deg2rad(0),
                                            np.deg2rad(45),
                                            mag=Hmag),
                             silent=True)
    model.adam_gradient_descent(max_steps=1e9, tol=1e-8, learning_rate=1e-5)
    mvector = VectorObj.from_spherical(model.history['theta_0'][-1],
                                       model.history['phi_0'][-1])
    ms.append(mvector)
    fs.append(model.get_fmr(0))

In [ ]:
with plt.style.context(['science', 'nature']):
    fig, ax = plt.subplots(dpi=300)
    ax2 = ax.twinx()
    ax2.plot(Hspace / 1e3, np.asarray(fs)/1e3, color='crimson', label='f')
    ax.plot(
        Hspace / 1e3,
        np.asarray(ms)[:, 2],
        
    )

    ax.set_xlabel("Field [A/m]")
    ax.set_ylabel("m [a.u.]")
    ax2.set_ylabel("Frequency [GHz]")
    ax2.legend()